<a href="https://colab.research.google.com/github/utkarsh8273/FACIAL_EXPRESSION/blob/main/Facial_Feature_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import glob
import cv2  

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
!unzip "gdrive/My Drive/facial/data.zip"

Archive:  gdrive/My Drive/facial/data.zip
replace Dataset/labels.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
path1 = glob.glob("Dataset/train/surprise/*.png")
X_train = np.zeros((31806,48,48))
Y_train = np.zeros((31806,1))
k = 0
for i in path1 :
  img = cv2.imread(i,0)
  #print(type(img))
  X_train[k,:,:] = np.copy(img)
  Y_train[k,:] = 0
  k = k + 1
  print(k)
path2 = glob.glob("Dataset/train/anger/*.png")
for i in path2 :
  img = cv2.imread(i,0)
  #print(type(img))
  X_train[k,:,:] = np.copy(img)
  Y_train[k,:] = 1
  k = k + 1
  print(k)
path3 = glob.glob("Dataset/train/happiness/*.png")
for i in path3 :
  img = cv2.imread(i,0)
  #print(type(img))
  X_train[k,:,:] = np.copy(img)
  Y_train[k,:] = 2
  k = k + 1
  print(k)
path4 = glob.glob("Dataset/train/neutral/*.png")
for i in path4 :
  img = cv2.imread(i,0)
  #print(type(img))
  X_train[k,:,:] = np.copy(img)
  Y_train[k,:] = 3
  k = k + 1
  print(k)
path5 = glob.glob("Dataset/train/fear/*.png")
for i in path5 :
  img = cv2.imread(i,0)
  #print(type(img))
  X_train[k,:,:] = np.copy(img)
  Y_train[k,:] = 4
  k = k + 1
  print(k)
path6 = glob.glob("Dataset/train/sadness/*.png")
for i in path6 :
  img = cv2.imread(i,0)
  #print(type(img))
  X_train[k,:,:] = np.copy(img)
  Y_train[k,:] = 5
  k = k + 1
  print(k)

In [ ]:
X_train = X_train/255

In [ ]:
X_train = np.reshape(X_train,(31806,48*48)).T
Y_train = Y_train.T
print(X_train.shape)
print(Y_train.shape)

(2304, 31806)
(1, 31806)


In [ ]:
shape = (31806, 6)
one_hot = np.zeros(shape)
rows = np.arange(31806)
one_hot[rows, Y_train.astype(int)] = 1
Y_train = one_hot.T
shuffle_index = np.random.permutation(X_train.shape[1])
X_train,Y_train = X_train[:,shuffle_index] , Y_train[:,shuffle_index]
print(X_train.shape)
print(Y_train.shape)
n = X_train.shape[0]
m = X_train.shape[1]

(2304, 31806)
(6, 31806)


In [ ]:
print(Y_train)

[[1. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
def softmax(x) :
  x = x - np.max(x)
  return np.exp(x) / np.sum(np.exp(x), axis=0,keepdims=True)
def relu(X) :
  X[X<0]=0
  return X
def initialize_wts(struct) :
  W = {}
  b = {}
  for i in range(1,len(struct)) :
    W["W" + str(i)] = np.random.randn(struct[i],struct[i-1])*np.sqrt(2/struct[i-1])
    #W["W" + str(i)] = np.random.randn(struct[i],struct[i-1])*0.01
    #W["W" + str(i)] = np.random.rand(struct[i],struct[i-1])
    b["b" + str(i)] = np.zeros((struct[i],1))
  return W,b
def forward_prop(X_train,W,b) :
  return relu(np.dot(W,X_train) + b)

In [ ]:
def model(struct,X_train,Y_train,lr,W,b,num_itr) :
  n = X_train.shape[0]
  m = X_train.shape[1]
  
  cost1 = 50
  for k in range(0,num_itr) :
    A = {}
    A["A" + str(0)] = np.copy(X_train)
    for i in range(1,len(struct)-1) :
      A["A" + str(i)] = forward_prop(A["A" + str(i-1)],W["W" + str(i)],b["b" + str(i)])
    A["A" + str(len(struct) - 1)] = softmax(np.dot(W["W" + str(len(struct) - 1)] , A["A" + str(len(struct) - 2)]) + b["b" + str(len(struct) - 1)])
    dW = {}
    db = {}
    dZ = {}
    dZ["dZ" + str(len(struct) - 1)] = A["A" + str(len(struct) - 1)] - Y_train
    dW["dW" + str(len(struct) - 1)] = (1/m)*(np.dot(dZ["dZ" + str(len(struct) - 1)],A["A" + str(len(struct) - 2)].T))
    db["db" + str(len(struct) - 1)] = (1/m)*(np.sum(dZ["dZ" + str(len(struct) - 1)],axis = 1, keepdims = True))
    for i in range(0,len(W) - 1) :
      dZ["dZ" + str(len(W) - 1 - i)] = np.dot(W["W" + str(len(W) - i)].T,dZ["dZ" + str(len(W) - i)])*np.int64(A["A" + str(len(W) - 1 - i)] > 0)
      dW["dW" + str(len(W) - 1 - i)] = (1/m)*np.dot(dZ["dZ" + str(len(W) - 1 - i)] , A["A" + str(len(W) - 1 - i - 1)].T)
      db["db" + str(len(W) - i - 1)] = (1/m)*np.sum(dZ["dZ" + str(len(W) - 1 - i)],axis = 1,keepdims = True)
    for i in range(1,len(W) + 1) :
      W["W" + str(i)] = W["W" + str(i)] - lr*dW["dW" + str(i)]
      b["b" + str(i)] = b["b" + str(i)] - lr*db["db" + str(i)]
    #cost1 = (-1/m)*np.sum(np.dot(Y_train,np.log(A["A" + str(len(struct) - 1)]).T) + np.dot(1 - Y_train,np.log(1 - A["A" + str(len(struct) - 1)]).T))
    cost1 = -np.mean(Y_train * np.log(A["A" + str(len(struct) - 1)] + 1e-8))
    #log_likelihood = -np.log(A["A" + str(len(struct) - 1)][range(m),Y_train])
    #cost1 = np.sum(log_likelihood) / m
    #costu.append(float(cost1))
    #print("Cost after {}th iteration is : {} {}".format(k,cost1,lr))
  #plt.plot(list(range(num_itr)) , costu)
  #plt.show()
  return W,b,A["A" + str(len(struct) - 1)],float(cost1)

In [ ]:
def train(struct,X_train,Y_train,lr,epoks,batch_size):
  costu = []
  W,b = initialize_wts(struct)
  for j in range(0,epoks) :
    for i in range(0,int(X_train.shape[1]/batch_size)+1) :
      if i != int(X_train.shape[1]/batch_size) :
        X_batch = X_train[:,i*batch_size:(i+1)*batch_size]
        Y_batch = Y_train[:,i*batch_size:(i+1)*batch_size]
        lim = (i+1)*batch_size
      elif i*batch_size == X_train.shape[1] :
        continue
      else :
        X_batch = X_train[:,i*batch_size:-1]
        Y_batch = Y_train[:,i*batch_size:-1]
        lim = X_train.shape[1]
      W,b,A,cost1 = model(struct , X_batch , Y_batch , lr ,W,b,num_itr = 1)
      costu.append(cost1)
      print("Epok : {}  Data_Passed : {}/{}  Train_Loss : {}".format(j+1,lim,X_train.shape[1],cost1))
  A = {}
  A["A" + str(0)] = np.copy(X_train)
  for i in range(1,len(struct)-1) :
    A["A" + str(i)] = forward_prop(A["A" + str(i-1)],W["W" + str(i)],b["b" + str(i)])
  A = softmax(np.dot(W["W" + str(len(struct) - 1)] , A["A" + str(len(struct) - 2)]) + b["b" + str(len(struct) - 1)])
  plt.plot(list(range(len(costu))),costu)
  return W,b,A

In [ ]:
W,b,A = train([n,1152,500,100,6],X_train,Y_train,0.1,15,64)

In [ ]:
def get_output(X,W,b) :
  struct = len(W) + 1
  A = {}
  A["A" + str(0)] = np.copy(X)
  for i in range(1,struct-1) :
    A["A" + str(i)] = forward_prop(A["A" + str(i-1)],W["W" + str(i)],b["b" + str(i)])
  A["A" + str(struct - 1)] = softmax(np.dot(W["W" + str(struct - 1)] , A["A" + str(struct - 2)]) + b["b" + str(struct - 1)])
  return A["A" + str(struct - 1)]

In [ ]:
def predict(A,Y) :
  for i in range(0,A.shape[1]) :
    a = np.max(A[:,i])
    A[:,i] = (A[:,i] == a)
  fai = np.absolute(A - Y)
  count = 0
  for i in range(0,fai.shape[1]) :
    if np.sum(fai[:,i]) == 0 :
      count += 1
  return (count/A.shape[1])*100

In [ ]:
A_train = get_output(X_train,W,b)
#A_test  = get_output(X_test,W,b)
print("Train accuracy is {}".format(predict(A_train,Y_train)))
#print("Test accuracy is {}".format(predict(A_test,Y_test)))

Train accuracy is 45.78381437464629
